<a href="https://colab.research.google.com/github/jumpingsphinx/loan_data/blob/main/main_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is the main file of the code I used on this analysis. However, there are a few files used as support for visualization generation, statistical testing, as well as a few key graphical visualizations that can be found on my github repo at [this link.](https://github.com/jumpingsphinx/loan_data)

Additionally, you might need to pip install some of the libraries, and mount your drive for this to work. I ran this notebook locally in VSCode and then uploaded it to Colab, so there might be a few tweaks that need to be made for it to run.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score, classification_report,
                           confusion_matrix, roc_auc_score, roc_curve)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pickle

In [ ]:
def load_and_examine_data(filepath):
    """
    Load the data and perform initial examination
    """
    # Read the data
    df = pd.read_csv(filepath)

    # Basic information about the dataset
    print("\nDataset Overview:")
    print(f"Number of loans: {len(df)}")
    print(f"Number of features: {len(df.columns)}")

    return df

def clean_monetary_columns(df):
    """
    Clean monetary columns by removing '$', ',' with regex and converting to float for analysis
    """
    money_columns = ['DISBURSEMENT_AMOUNT', 'BALANCE_AMOUNT', 'CHARGE_OFF_AMOUNT',
                    'LOAN_AMOUNT', 'SBA_APPROVED_AMOUNT']

    for col in money_columns:
        df[col] = df[col].str.replace('[\$,\s]', '', regex=True).astype(float)

    print("\nMonetary Columns Statistics:")
    print(df[money_columns].describe())

    return df

def process_dates(df):
    """
    Convert date columns to datetime and create derived features
    """
    date_columns = ['APPROVAL_DATE', 'DEFAULT_DATE', 'DISBURSEMENT_DATE']

    for col in date_columns:
        df[col] = pd.to_datetime(df[col], format='%d-%b-%y', errors='coerce')

    # Create time-based features
    df['DAYS_TO_DISBURSEMENT'] = (df['DISBURSEMENT_DATE'] - df['APPROVAL_DATE']).dt.days

    # Certain loans had approval years different to their approval date; we'll assume for this analysis that the approval date was correct and overwrite that.
    df['APPROVAL_YEAR'] = df['APPROVAL_DATE'].dt.year
    df['APPROVAL_MONTH'] = df['APPROVAL_DATE'].dt.month

    # Calculate days to default for defaulted loans
    df['DAYS_TO_DEFAULT'] = (df['DEFAULT_DATE'] - df['DISBURSEMENT_DATE']).dt.days

    # Create default flag
    df['DEFAULT_FLAG'] = df['DEFAULT_DATE'].notna().astype(int)

    print("\nTime-based Features Statistics:")
    print(df[['DAYS_TO_DISBURSEMENT', 'DAYS_TO_DEFAULT']].describe())

    return df

def analyze_categorical_features(df):
    """
    Analyze interesting categorical features and their relationships with defaults
    """
    categorical_cols = ['IS_NEW', 'IS_URBAN',
                       'IS_REVOLVER', 'IS_LOW_DOC']

    print("\nCategorical Features Analysis:")
    for col in categorical_cols:
        print(f"\n{col} Distribution:")
        print(df[col].value_counts(normalize=True).head())

        # Calculate default rates by category
        default_rates = df.groupby(col)['DEFAULT_FLAG'].mean()
        print(f"\nDefault Rates by {col}:")
        print(default_rates)

        # Print number of unique values
        print(f"\nNumber of unique {col} values: {df[col].nunique()}")

    return df

def analyze_industry_patterns(df):
    """
    Analyze patterns in industry codes and their relationship with defaults
    """
    # Extract industry sectors (first 2 digits)
    df['INDUSTRY_SECTOR'] = df['INDUSTRY_ID'].astype(str).str[:2]

    # Calculate metrics by industry sector
    industry_metrics = df.groupby('INDUSTRY_SECTOR').agg({
        'LOAN_AMOUNT': 'mean',
        'DEFAULT_FLAG': 'mean',
        'CHARGE_OFF_AMOUNT': 'mean',
        'EMPLOYEE_COUNT': 'mean'
    }).round(2)

    print("\nIndustry Sector Analysis:")
    print("\nIndustries most likely to default:")
    print(industry_metrics.sort_values('DEFAULT_FLAG', ascending=False).head(10))
    print("\nIndustries least likely to default:")
    print(industry_metrics.sort_values('DEFAULT_FLAG', ascending=True).head(10))

    return df

def analyze_economic_indicators(df):
    """
    Analyze relationships between economic indicators and loan performance
    """
    economic_cols = ['TREASURY_YIELD', 'CPI_INDEX', 'GDP', 'MORTGAGE_30_US_FIXED',
                    'UNRATE', 'INDPRO_INDEX', 'UMCSENT_INDEX', 'CSUSHPINSA_INDEX',
                    'CP_INDEX', 'FEDFUNDS_RATE']

    # Calculate correlations with default
    correlations = pd.DataFrame({
        'correlation': [df[col].corr(df['DEFAULT_FLAG']) for col in economic_cols]
    }, index=economic_cols)

    print("\nEconomic Indicators Correlation with Defaults:")
    print(correlations.sort_values('correlation', ascending=False))

    return df

def create_derived_features(df):
    """
    Create derived features for modeling
    """
    # Loan characteristics
    df['LOAN_PER_EMPLOYEE'] = df['LOAN_AMOUNT'] / df['EMPLOYEE_COUNT'].replace(0, 1)
    df['SBA_GUARANTEE_RATIO'] = df['SBA_APPROVED_AMOUNT'] / df['LOAN_AMOUNT']

    # Business impact
    df['TOTAL_JOBS_IMPACT'] = df['JOBS_CREATED_COUNT'] + df['JOBS_RETAINED_COUNT']
    df['JOBS_IMPACT_RATIO'] = df['TOTAL_JOBS_IMPACT'] / df['EMPLOYEE_COUNT'].replace(0, 1)

    # Risk indicators
    df['CHARGE_OFF_RATIO'] = df['CHARGE_OFF_AMOUNT'] / df['LOAN_AMOUNT']

    print("\nDerived Features Statistics:")
    derived_features = ['LOAN_PER_EMPLOYEE', 'SBA_GUARANTEE_RATIO', 'TOTAL_JOBS_IMPACT',
                       'JOBS_IMPACT_RATIO', 'CHARGE_OFF_RATIO']
    print(df[derived_features].describe())

    return df

def identify_outliers(df):
    """
    Identify outliers in key numeric features using Tukey's fence method
    """
    numeric_cols = ['LOAN_AMOUNT', 'EMPLOYEE_COUNT', 'TERM', 'JOBS_CREATED_COUNT',
                   'JOBS_RETAINED_COUNT']

    print("\nOutlier Analysis:")
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
        print(f"\n{col} outliers: {len(outliers)} ({len(outliers)/len(df):.2%})")
        print(f"Range: {df[col].min():.2f} to {df[col].max():.2f}")

    return df

def prepare_modeling_data(df):
    """
    Prepare final datasets for modeling
    """
    # Select features for modeling
    numeric_features = ['TERM', 'EMPLOYEE_COUNT', 'JOBS_CREATED_COUNT',
                       'JOBS_RETAINED_COUNT', 'LOAN_AMOUNT', 'SBA_APPROVED_AMOUNT',
                       'TREASURY_YIELD', 'UNRATE', 'GDP', 'LOAN_PER_EMPLOYEE',
                       'SBA_GUARANTEE_RATIO', 'TOTAL_JOBS_IMPACT']

    categorical_features = ['STATE', 'BANK_STATE', 'INDUSTRY_SECTOR', 'IS_NEW',
                          'IS_URBAN', 'IS_REVOLVER', 'IS_LOW_DOC']

    # Create modeling dataset
    modeling_data = df[numeric_features + categorical_features].copy()
    modeling_data = pd.get_dummies(modeling_data, columns=categorical_features)

    # Create target variables
    targets = {
        'default': df['DEFAULT_FLAG'],
        'charge_off': df['CHARGE_OFF_AMOUNT']
    }

    print("\nModeling Data Shape:", modeling_data.shape)
    print("Number of features:", len(modeling_data.columns))

    return modeling_data, targets

def main_eda(filepath):
    """
    Main function to run the complete EDA pipeline
    """
    # Load and examine data
    print("Loading and examining data...")
    df = load_and_examine_data(filepath)

    # Clean and preprocess
    print("\nCleaning monetary columns...")
    df = clean_monetary_columns(df)

    print("\nProcessing dates...")
    df = process_dates(df)

    # Analyze patterns
    print("\nAnalyzing categorical features...")
    df = analyze_categorical_features(df)

    print("\nAnalyzing industry patterns...")
    df = analyze_industry_patterns(df)

    print("\nAnalyzing economic indicators...")
    df = analyze_economic_indicators(df)

    # Create features and identify outliers
    print("\nCreating derived features...")
    df = create_derived_features(df)

    print("\nIdentifying outliers...")
    df = identify_outliers(df)

    # Prepare modeling data
    print("\nPreparing modeling data...")
    modeling_data, targets = prepare_modeling_data(df)

    return df, modeling_data, targets

In [ ]:
filepath = r"C:\Users\ashwi\Downloads\loan_data.csv"  # Replace with your file path
df, modeling_data, targets = main_eda(filepath)

In [ ]:
def merge_economic_data(loan_df, economic_indicators_path):
    """
    Merges loan data with economic indicators based on closest dates and handles missing values.

    Parameters:
    loan_data_path (str): Path to the loan data CSV
    economic_indicators_path (str): Path to the economic indicators CSV

    Returns:
    pd.DataFrame: Merged dataset with handled missing values
    """
    # Read the datasets

    econ_df = pd.read_csv(economic_indicators_path)

    # Convert approval dates to datetime
    loan_df['APPROVAL_DATE'] = pd.to_datetime(loan_df['APPROVAL_DATE'], format='%d-%b-%y')

    # Convert economic indicators index to datetime
    econ_df['DATE'] = pd.to_datetime(econ_df.iloc[:, 0])

    # Create a function to find closest date match
    def find_closest_date(target_date):
        return abs(econ_df['DATE'] - target_date).idxmin()

    # Create new columns for matched economic indicators
    economic_columns = ['CPI_INDEX', 'GDP', 'MORTGAGE_30_US_FIXED', 'UNRATE',
                       'INDPRO_INDEX', 'UMCSENT_INDEX', 'CSUSHPINSA_INDEX',
                       'CP_INDEX', 'FEDFUNDS_RATE']

    # Initialize new columns with existing values
    for col in economic_columns:
        if col in loan_df.columns:
            loan_df[f'{col}_matched'] = loan_df[col]
        else:
            loan_df[f'{col}_matched'] = np.nan

    # Match economic indicators based on closest date
    for idx, row in loan_df.iterrows():
        closest_idx = find_closest_date(row['APPROVAL_DATE'])
        for col in economic_columns:
            # Only fill if the value is missing or NaN
            if pd.isna(loan_df.at[idx, f'{col}_matched']):
                loan_df.at[idx, f'{col}_matched'] = econ_df.at[closest_idx, col]

    # Replace original columns with matched values where missing
    for col in economic_columns:
        if col in loan_df.columns:
            loan_df[col] = loan_df[col].fillna(loan_df[f'{col}_matched'])
        else:
            loan_df[col] = loan_df[f'{col}_matched']

        # Drop the temporary matched columns
        loan_df = loan_df.drop(f'{col}_matched', axis=1)

    return loan_df

df = merge_economic_data(df, r"C:\dev\python\loan_data\fred_data\economic_indicators_1960_2015.csv") # Replace with your file path

In [ ]:
def prepare_regression_features(df):
    """
    Prepare features for modeling with detailed NaN checking to ensure we lose as few rows as possible
    """
    economic_indicators = [
        'TREASURY_YIELD', 'UNRATE', 'GDP', 'MORTGAGE_30_US_FIXED',
        'INDPRO_INDEX', 'UMCSENT_INDEX', 'CSUSHPINSA_INDEX',
        'CP_INDEX', 'FEDFUNDS_RATE'
    ]

    # Create a copy of the dataframe
    df_processed = df.copy()
    df['Processed'] = (df_processed['LOAN_STATUS'] == 'CHGOFF').astype(int)

    # Fill in the economic indicators based on APPROVAL_YEAR
    yearly_indicators = df.groupby('APPROVAL_YEAR')[economic_indicators].mean()
    for indicator in economic_indicators:
        df_processed[indicator] = df_processed['APPROVAL_YEAR'].map(yearly_indicators[indicator])

    # Drop rows with NaN in economic indicators (FRED data is not available for all years and therefore some rows may have NaN)
    print(f"\nNumber of rows before dropping NaN: {len(df_processed)}")
    df_processed = df_processed.dropna(subset=economic_indicators)
    print(f"Number of rows after dropping NaN: {len(df_processed)}")

    # Select features
    numeric_features = [
        'TERM', 'EMPLOYEE_COUNT', 'JOBS_CREATED_COUNT', 'JOBS_RETAINED_COUNT',
        'LOAN_AMOUNT', 'SBA_APPROVED_AMOUNT', 'LOAN_AMOUNT', 'SBA_APPROVED_AMOUNT'
    ] + economic_indicators

    categorical_features = [
        'STATE', 'BANK_STATE', 'INDUSTRY_SECTOR', 'IS_NEW',
        'IS_URBAN', 'IS_REVOLVER', 'IS_LOW_DOC'
    ]

    # Check categorical features for NaN
    print("\nChecking categorical features for NaN:")
    print(df_processed[categorical_features].isna().sum())

    # Handle numeric features
    df_numeric = df_processed[numeric_features].copy()

    # Handle categorical features
    # Fill NaN in categorical features before encoding
    for cat_feature in categorical_features:
        df_processed[cat_feature] = df_processed[cat_feature].fillna('Missing')

    X_categorical = pd.get_dummies(df_processed[categorical_features], drop_first=True)

    # Combine features
    X = pd.concat([df_numeric, X_categorical], axis=1)

    # Check for NaN in final feature matrix
    print("\nChecking combined feature matrix for NaN:")
    nan_cols = X.columns[X.isna().any()].tolist()
    if nan_cols:
        print("Columns with NaN values:", nan_cols)
        print("NaN counts in these columns:")
        print(X[nan_cols].isna().sum())

        # Drop any remaining rows with NaN
        X = X.dropna()
        print(f"\nShape after dropping all NaN: {X.shape}")
    else:
        print("No NaN values found in combined feature matrix")

    # Target variable
    y = df_processed['CHARGE_OFF_AMOUNT'].fillna(0)
    scaler_y = StandardScaler()
    y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

    # Make sure y aligns with X after any row dropping
    y = y[X.index]

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_scaled, test_size=0.2, random_state=42
    )

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print("\nFeature preparation completed:")
    print(f"Training set shape: {X_train_scaled.shape}")
    print(f"Test set shape: {X_test_scaled.shape}")

    return X_train_scaled, X_test_scaled, y_train, y_test, X.columns, scaler, scaler_y

def train_random_forest(X_train_scaled, X_test_scaled, y_train, y_test):
    """
    Train Random Forest regression model and evaluate its performance.
    Initially, evaluated performance of Gradient Boosting, L1 and L2 regularized linear models, but Random Forest performed the best on testing and trained data.
    """
    # Initialize and train model
    model = RandomForestRegressor(n_estimators=300, random_state=7, verbose=1, n_jobs=12)
    model.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = model.predict(X_test_scaled)

    # Calculate metrics
    results = {
        'R2': r2_score(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred)
    }

    # Print results
    print("\nRandom Forest Results:")
    print(f"R² Score: {results['R2']:.4f}")
    print(f"RMSE: {results['RMSE']:.4f}")
    print(f"MAE: {results['MAE']:.4f}")

    return results, model

def analyze_feature_importance(model, feature_names):
    """
    Analyze feature importance for Random Forest model
    """
    importance = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 10 Most Important Features:")
    print(importance.head(10))

    return importance

def plot_regression_results(model, X_test_scaled, y_test, feature_importance):
    """
    Create visualizations of model performance
    """
    # Make predictions
    y_pred = model.predict(X_test_scaled)

    # Create figure with multiple subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Actual vs Predicted
    axes[0, 0].scatter(y_test, y_pred, alpha=0.5)
    axes[0, 0].plot([y_test.min(), y_test.max()],
                   [y_test.min(), y_test.max()],
                   'r--', lw=2)
    axes[0, 0].set_title('Actual vs Predicted Values')
    axes[0, 0].set_xlabel('Actual Charge-off Amount')
    axes[0, 0].set_ylabel('Predicted Charge-off Amount')

    # 2. Residuals Plot
    residuals = y_test - y_pred
    axes[0, 1].scatter(y_pred, residuals, alpha=0.5)
    axes[0, 1].axhline(y=0, color='r', linestyle='--')
    axes[0, 1].set_title('Residuals Plot')
    axes[0, 1].set_xlabel('Predicted Values')
    axes[0, 1].set_ylabel('Residuals')

    # 3. Feature Importance Plot
    importance = feature_importance.head(10)
    sns.barplot(x='importance', y='feature', data=importance, ax=axes[1, 0])
    axes[1, 0].set_title('Top 10 Feature Importance')

    # 4. Prediction Error Distribution
    sns.histplot(residuals, kde=True, ax=axes[1, 1])
    axes[1, 1].set_title('Distribution of Prediction Errors')
    axes[1, 1].set_xlabel('Prediction Error')

    plt.tight_layout()
    return fig

def cross_validate_model(model, X_train_scaled, y_train, cv=5):
    """
    Perform cross-validation
    """
    cv_scores = cross_val_score(
        model, X_train_scaled, y_train,
        cv=cv, scoring='r2',
        n_jobs=12, verbose=1
    )

    print("\nCross-validation Results:")
    print(f"Average R2: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

    return cv_scores

def main_regression(df):
    """
    Main function to run the Random Forest regression analysis
    """
    # Prepare features
    print("Preparing features...")
    X_train_scaled, X_test_scaled, y_train, y_test, feature_names, scaler, scaler_y = prepare_regression_features(df)

    # Train Random Forest model
    print("\nTraining Random Forest model...")
    results, model = train_random_forest(X_train_scaled, X_test_scaled, y_train, y_test)

    # Analyze feature importance
    print("\nAnalyzing feature importance...")
    importance = analyze_feature_importance(model, feature_names)

    # Cross-validate model
    print("\nPerforming cross-validation...")
    cv_scores = cross_validate_model(model, X_train_scaled, y_train)

    # Create visualizations
    print("\nCreating visualizations...")
    fig = plot_regression_results(model, X_test_scaled, y_test, importance)

    print("\nSaving model to disk...")
    with open('random_forest_regression_model.pkl', 'wb') as file:
        pickle.dump({
            'model': model,
            'feature_names': feature_names,
            'scaler': scaler,
            'scaler_y': scaler_y
        }, file)
    print("Model saved successfully!")

    return results, model, importance, fig, (X_train_scaled, X_test_scaled, y_train, y_test)

In [ ]:
results_df, best_model, importance, fig, (X_train_scaled, X_test_scaled, y_train, y_test) = main_regression(df)

In [ ]:
def prepare_classification_data(df):
    """
    Prepare already cleaned data for classification.
    Drops high cardinality columns and encodes remaining categorical variables.
    """
    # Drop high cardinality columns and columns that would be unnecessary or unhelpful for classification, or are otherwise redundant
    columns_to_drop = ['BORROWER_NAME', 'CITY', 'STATE', 'BANK', 'BANK_STATE',
                      'LOAN_STATUS', 'DEFAULT_DATE', 'CHARGE_OFF_AMOUNT',
                      'DISBURSEMENT_DATE', 'BALANCE_AMOUNT', 'BALANCE_AMOUNT',
                      'APPROVAL_DATE', 'Processed', 'DEFAULT_FLAG', 'DAYS_TO_DEFAULT',
                      'CHARGE_OFF_RATIO']

    df_cleaned = df.drop(columns=columns_to_drop)

    # Identify remaining categorical columns
    categorical_columns = df_cleaned.select_dtypes(include=['object']).columns
    print("\nCategorical columns to encode:", categorical_columns.tolist())

    # Create dummy variables for remaining categorical features
    X = pd.get_dummies(df_cleaned, columns=categorical_columns, drop_first=True)

    # Create target variable (1 for CHGOFF, 0 for PIF)
    y = (df['LOAN_STATUS'] == 'CHGOFF').astype(int)

    print("\nData Preparation Summary:")
    print(df['LOAN_STATUS'].value_counts())
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of samples: {X.shape[0]}")
    print(f"Target distribution:\n{y.value_counts(normalize=True)}")

    return X, y, X.columns.tolist()

def train_random_forest_classifier(X_train_scaled, X_test_scaled, y_train, y_test):
    """
    Train Random Forest model and evaluate its performance
    """
    # Initialize and train model
    model = RandomForestClassifier(n_estimators=300, random_state=7, n_jobs=12, verbose=1)
    print("\nTraining Random Forest...")
    model.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

    # Calculate metrics
    results = {
        'accuracy': model.score(X_test_scaled, y_test),
        'roc_auc': roc_auc_score(y_test, y_pred_proba),
        'classification_report': classification_report(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }

    print("\nRandom Forest Results:")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"ROC AUC: {results['roc_auc']:.4f}")
    print("\nClassification Report:")
    print(results['classification_report'])

    return results, model

def analyze_feature_importance(model, feature_names, X, y):
    """
    Analyze feature importance for Random Forest model
    """
    importance = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 10 Most Important Features:")
    print(importance.head(10))

    X_df = pd.DataFrame(X, columns=feature_names)
    correlations = pd.DataFrame({
        'feature': feature_names,
        'correlation': [X_df[col].corr(y) for col in feature_names]
    }).sort_values('correlation', ascending=False)

    print("\nTop 10 Positive Correlations with Default:")
    print(correlations.head(10))

    print("\nTop 10 Negative Correlations with Default:")
    print(correlations.tail(10))

    return importance, correlations

def plot_results(model, results, X_test_scaled, y_test, feature_names, X, y):
    """
    Create visualizations of model performance including correlations
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Feature Importance from Random Forest
    importance, correlations = analyze_feature_importance(model, feature_names, X, y)
    top_features = importance.head(10)
    sns.barplot(x='importance', y='feature', data=top_features, ax=axes[0, 0])
    axes[0, 0].set_title('Top 10 Feature Importance (Random Forest)')

    # 2. Feature Correlations
    correlations_plot = pd.concat([
    correlations.nlargest(5, 'correlation'),
    correlations.nsmallest(5, 'correlation')
])
    sns.barplot(x='correlation', y='feature', data=correlations_plot, ax=axes[0, 1],
               palette=['red' if x < 0 else 'blue' for x in correlations_plot['correlation']])
    axes[0, 1].set_title('Top 5 Positive and Negative Correlations')

    # 3. Confusion Matrix
    y_pred = model.predict(X_test_scaled)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 0])
    axes[1, 0].set_title('Confusion Matrix')
    axes[1, 0].set_xlabel('Predicted')
    axes[1, 0].set_ylabel('Actual')

    # 4. ROC Curve
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    axes[1, 1].plot(fpr, tpr, label=f'Random Forest (AUC = {results["roc_auc"]:.3f})')
    axes[1, 1].plot([0, 1], [0, 1], 'r--')
    axes[1, 1].set_xlabel('False Positive Rate')
    axes[1, 1].set_ylabel('True Positive Rate')
    axes[1, 1].set_title('ROC Curve')
    axes[1, 1].legend()

    plt.tight_layout()
    return fig, importance, correlations

def cross_validate_model(model, X_train_scaled, y_train, cv=5):
    """
    Perform cross-validation
    """
    cv_scores = cross_val_score(
        model, X_train_scaled, y_train,
        cv=cv, scoring='roc_auc',
        n_jobs=12, verbose=1
    )

    print("\nCross-validation Results:")
    print(f"Average ROC AUC: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

    return cv_scores

def save_classification_model(model, feature_names, scaler=None, filename='random_forest_classifier_model.pkl'):
    """
    Save the trained classification model and associated data to disk

    Parameters:
    model: The trained Random Forest classifier
    feature_names: List of feature names used in training
    scaler: The StandardScaler used to scale features (if any)
    filename: Name of the file to save the model to
    """
    print(f"\nSaving model to {filename}...")
    try:
        save_dict = {
            'model': model,
            'feature_names': feature_names,
        }
        if scaler is not None:
            save_dict['scaler'] = scaler

        with open(filename, 'wb') as file:
            pickle.dump(save_dict, file)
        print("Model saved successfully!")
    except Exception as e:
        print(f"Error saving model: {str(e)}")

def run_classification(X, y):
    """
    Main function to run Random Forest classification analysis on preprocessed data
    """

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=7, stratify=y
    )

    print("\nClass distribution in training set:")
    print(y_train.value_counts(normalize=True))

    # Train Random Forest model
    print("\nTraining model...")
    results, model = train_random_forest_classifier(X_train, X_test, y_train, y_test)

    # Analyze feature importance
    print("\nAnalyzing feature importance...")
    importance = analyze_feature_importance(model, X.columns, X_train, y_train)

    # Cross-validate model
    print("\nPerforming cross-validation...")
    cv_scores = cross_validate_model(model, X_train, y_train)

    # Create visualizations
    print("\nCreating visualizations...")
    fig, importance, correlations = plot_results(model, results, X_test, y_test, X.columns, X, y)

    # Save model - only do if you want to save the model you train locally
    # print("\nSaving model...")
    # save_classification_model(model, X.columns)
    return results, model, importance, correlations, fig, (X_train, X_test, y_train, y_test)


In [ ]:
# Prepare data for classification and run classification model
X, y, feature_names = prepare_classification_data(df)
results, best_model, importance, correlations, fig, (X_train, X_test, y_train, y_test) = run_classification(X, y)